# Résolution de Sudoku avec OR-Tools

## Introduction

Ce projet a pour objectif de résoudre des grilles de Sudoku en utilisant différentes méthodes d'optimisation fournies par la bibliothèque OR-Tools de Google. OR-Tools propose des solveurs pour les problèmes de satisfaction de contraintes (CSP), la programmation linéaire mixte (MIP), et bien d'autres méthodes optimisées pour les problèmes d'optimisation.

Nous allons explorer plusieurs implémentations de solveurs de Sudoku, les tester sur des grilles de différentes difficultés et comparer leurs performances.

## Installation de OR-Tools

Pour utiliser OR-Tools avec C#, nous devons d'abord installer la bibliothèque. Vous pouvez suivre les instructions sur le site officiel de [Google OR-Tools](https://developers.google.com/optimization).

### Installation de OR-Tools


In [1]:
#r "nuget: Google.OrTools"

Installed Packages Google.OrTools, 9.10.4067

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [2]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

## Implémentations des Solveurs

### Solveur par contraintes (CpSolver)

L'algorithme de satisfaction de contraintes (CSP) utilise les contraintes pour réduire l'espace de recherche et trouver une solution qui satisfait toutes les contraintes du problème. Nous utilisons `CpSolver` d'OR-Tools pour cette implémentation.


In [3]:
using Google.OrTools.ConstraintSolver;
using SimpleCPSolver = Google.OrTools.ConstraintSolver.Solver;


public class OrToolsCpSolver : ISudokuSolver
{
    private const int GridSize = 9;

    public SudokuGrid Solve(SudokuGrid s)
    {
        var solver = new SimpleCPSolver("CpSimple");

        IntVar[,] gridVars = new IntVar[GridSize, GridSize];
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                if (s.Cells[i, j] == 0)
                {
                    gridVars[i, j] = solver.MakeIntVar(1, 9, $"{i}{j}");
                }
                else
                {
                    gridVars[i, j] = solver.MakeIntConst(s.Cells[i, j]);
                }
            }
        }

        for (int i = 0; i < GridSize; i++)
        {
            IntVar[] col = new IntVar[9];
            IntVar[] row = new IntVar[9];

            for (int j = 0; j < GridSize; j++)
            {
                row[j] = gridVars[i, j];
                col[j] = gridVars[j, i];

                if (i % 3 == 1 && j % 3 == 1)
                {
                    IntVar[] square = new IntVar[9];
                    int x = 0;
                    for (int k = i - 1; k < i + 2; k++)
                    {
                        int y = 0;
                        for (int l = j - 1; l < j + 2; l++)
                        {
                            square[3 * x + y] = gridVars[k, l];
                            y++;
                        }
                        x++;
                    }
                    solver.Add(solver.MakeAllDifferent(square));
                }
            }
            solver.Add(solver.MakeAllDifferent(col));
            solver.Add(solver.MakeAllDifferent(row));
        }

        DecisionBuilder db = solver.MakePhase(gridVars.Flatten(), SimpleCPSolver.CHOOSE_FIRST_UNBOUND, SimpleCPSolver.ASSIGN_MIN_VALUE);

        solver.NewSearch(db);
        while (solver.NextSolution())
        {
            for (int row = 0; row < GridSize; row++)
            {
                for (int col = 0; col < GridSize; col++)
                {
                    s.Cells[row, col] = (int)gridVars[row, col].Value();
                }
            }
            break;
        }

        return s;
    }
}

### Tests du Solveur par contraintes

Nous allons tester ce solveur sur une grille de Sudoku.

In [4]:
var cpSolver = new OrToolsCpSolver();

var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Facile Initial:");
SudokuHelper.SolveSudoku(easySudoku, cpSolver);

var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Moyen Initial:");
SudokuHelper.SolveSudoku(mediumSudoku, cpSolver);

var hardSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Hard).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Difficile Initial:");
SudokuHelper.SolveSudoku(hardSudoku, cpSolver);


Puzzle Sudoku Facile Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 16,8549 ms

Puzzle Sudoku Moyen Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 8  5  9 | 6  1  2 | 4  3  7 | 
| 7  2  3 | 8  5  4 | 1  6  9 | 
| 1  6  4 | 3  7  9 | 5  2  8 | 
-------------------------------
| 9  8  6 | 1  4  7 | 3  5  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 2  4  1 | 5  9  3 | 7  8  6 | 
-------------------------------
| 4  3  2 | 9  8  1 | 6  7  5 | 
| 6  1  7 | 4  2  5 | 8  9  3 | 
| 5  9  8 | 7  3  6 | 2  4  1 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1,8515 ms

Puzzle Sudoku Difficile Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 2,6785 ms

### Solveur MIP (Mixed-Integer Programming)

L'algorithme de programmation linéaire mixte (MIP) combine la programmation linéaire et les variables entières pour résoudre des problèmes d'optimisation. Nous utilisons `Mixed-Integer Programming` d'OR-Tools pour cette implémentation.


In [5]:
using Google.OrTools.LinearSolver;
using LinearSolver = Google.OrTools.LinearSolver.Solver;

public class OrToolsMIPSolver : ISudokuSolver
{
    private const int GridSize = 9;

    public SudokuGrid Solve(SudokuGrid s)
    {
        LinearSolver solver = LinearSolver.CreateSolver("SCIP");
        if (solver == null)
        {
            throw new InvalidOperationException("Solver initialization failed.");
        }

        Variable[,,] cells = new Variable[GridSize, GridSize, GridSize];
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    cells[i, j, k] = solver.MakeIntVar(0, 1, $"Cell({i},{j},{k})");
                }
            }
        }

        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                if (s.Cells[i, j] != 0)
                {
                    solver.Add(cells[i, j, s.Cells[i, j] - 1] == 1);
                }
            }
        }

        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                var cellSum = new LinearExpr();
                for (int k = 0; k < GridSize; k++)
                {
                    cellSum += cells[i, j, k];
                }
                solver.Add(cellSum == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i++)
            {
                var rowSum = new LinearExpr();
                var colSum = new LinearExpr();
                for (int j = 0; j < GridSize; j++)
                {
                    rowSum += cells[i, j, k];
                    colSum += cells[j, i, k];
                }
                solver.Add(rowSum == 1);
                solver.Add(colSum == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int rowBlock = 0; rowBlock < 3; ++rowBlock)
            {
                for (int colBlock = 0; colBlock < 3; ++colBlock)
                {
                    var blockSum = new LinearExpr();
                    for (int i = 0; i < 3; i++)
                    {
                        for (int j = 0; j < 3; j++)
                        {
                            blockSum += cells[rowBlock * 3 + i, colBlock * 3 + j, k];
                        }
                    }
                    solver.Add(blockSum == 1);
                }
            }
        }

        var resultStatus = solver.Solve();
        if (resultStatus != LinearSolver.ResultStatus.OPTIMAL && resultStatus != LinearSolver.ResultStatus.FEASIBLE)
        {
            throw new Exception("No solution found.");
        }

        SudokuGrid solution = new SudokuGrid();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    if (cells[i, j, k].SolutionValue() == 1)
                    {
                        solution.Cells[i, j] = k + 1;
                    }
                }
            }
        }
        return solution;
    }
}


### Tests du Solveur MIP avec SCIP

Nous allons maintenant tester ce solveur sur une grille de Sudoku.

In [6]:
var mipSolver = new OrToolsMIPSolver();

var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).FirstOrDefault();
Console.WriteLine("Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Facile:");
SudokuHelper.SolveSudoku(easySudoku, mipSolver);

var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).FirstOrDefault();
Console.WriteLine("Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Moyen:");
SudokuHelper.SolveSudoku(mediumSudoku, mipSolver);

var hardSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Hard).FirstOrDefault();
Console.WriteLine("Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Difficile:");
SudokuHelper.SolveSudoku(hardSudoku, mipSolver);


Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Facile:


Résolution par le solver OrToolsMIPSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 20,7591 ms

Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Moyen:


Résolution par le solver OrToolsMIPSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 8  5  9 | 6  1  2 | 4  3  7 | 
| 7  2  3 | 8  5  4 | 1  6  9 | 
| 1  6  4 | 3  7  9 | 5  2  8 | 
-------------------------------
| 9  8  6 | 1  4  7 | 3  5  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 2  4  1 | 5  9  3 | 7  8  6 | 
-------------------------------
| 4  3  2 | 9  8  1 | 6  7  5 | 
| 6  1  7 | 4  2  5 | 8  9  3 | 
| 5  9  8 | 7  3  6 | 2  4  1 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 14,712 ms

Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Difficile:


Résolution par le solver OrToolsMIPSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 14,0888 ms